In [2]:
import tabula
import csv
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# URLs for the updated information
ns = "https://novascotiaimmigration.com/wp-content/uploads/Designated_AIP_employers.pdf"
nb = "https://www.welcomenb.ca/content/dam/wel-bien/pdf/atlantic_immigration_pilot_project_designated_employers_list.pdf"
pe = "https://www.princeedwardisland.ca/en/information/office-immigration/atlantic-immigration-pilot-designated-employers"
nl = "https://www.gov.nl.ca/immigration/immigrating-to-newfoundland-and-labrador/atlantic-immigration-pilot-program/designated-employers/"

In [3]:
# Newfoundland and Labrator

driver = webdriver.Chrome()
driver.get(nl)
time.sleep(3)
driver.execute_script("document.getElementsByName('DataTables_Table_0_length')[0][2].value = 1000;")
driver.execute_script("document.getElementsByName('DataTables_Table_0_length')[0][2].text = '1000';")
driver.find_element_by_xpath("//select[@name='DataTables_Table_0_length']/option[3]").click();

table = driver.page_source
driver.close()

nl_soup = BeautifulSoup(table, 'html.parser')
td_list_employers = nl_soup.findAll("td", class_="sorting_1")
td_list = nl_soup.findAll("td")
nl_address = []

In [4]:
nl_list = []

# Get the address
index = 1
while(index < len(td_list)):
    nl_address.append(td_list[index].text)
    index = index+5
    
# Get the employer name
index = 0
while(index < len(td_list_employers)):
    # Example of name: 
    #
    # useless o/a Name - Address
    nl_first_split = td_list_employers[index].text.split('o/a ')
    
    # If it has 'o/a' in the text
    if(len(nl_first_split) == 1):
        string = nl_first_split[0]
        print(nl_first_split)
    elif(len(nl_first_split) == 2):
        string =  nl_first_split[1]
        print(nl_first_split)
        
    if(string[0].isdigit()):
        nl_second_split = string.split(' ')
        if(len(nl_second_split)>1):
            index_ = 2
            name = nl_second_split[1] + ' '
            while(index_ != len(nl_second_split)):
                if(index_ !=len(nl_second_split)):
                    name = name + nl_second_split[index_]+' '
                else:
                    name = name + nl_second_split[index_]
                index_ = index_+1
            item = {"name": name, "address": nl_address[index]}
            nl_list.append(item)
    else:
        item = {"name": string, "address": nl_address[index]}
        nl_list.append(item)
    #print(nl_list[index])
    index = index+1

['10113 Newfoundland Limited ', 'Mary Brown’s']
['10162 NFLD Limited DBA ', 'Marsh Motor Sports']
['10464 Newfoundland Ltd. ', 'Jungle Jim’s Restaurant']
['10565 NFLD Inc. ', 'Hotel North']
['10566 NFLD Inc. ', 'Hotel North']
['10567 NFLD Inc. ', 'Hotel North']
['2554049 Ontario Inc. ', 'Extended Stay St. John’s']
['4411617 Canada Inc. ', 'Lasik MD']
['56955 Newfoundland and Labrador Corp ', 'Pleasantview Manor']
['61065 NL Inc. ', 'The Golden Years Estate']
['70535 Newfoundland and Labrador Inc.']
['70838 Newfoundland and Labrador Ltd. ', 'U Design']
['71036 Newfoundland and Labrador Ltd. ', 'Cabot Auto']
['74501 Newfoundland and Labrador Inc. ', 'St. John’s Fish Exchange']
['75961 Newfoundland & Labrador Inc. ', 'Cyno']
['77711 Newfoundland and Labrador Inc.']
['77727 Newfoundland and Labrador Inc. ', 'Burritos Mexican Grill']
['78117 Newfoundland and Labrador Limited']
['78187 Newfoundland & Labrador Inc. ', 'Mucho Burrito Avalon Mall']
['79109 Newfoundland & Labrador Inc. ', 'World

In [5]:
# Nova Scotia

tabula.convert_into(ns, "ns.csv", output_format="csv", pages='all')

# New Brunswick

tabula.convert_into(nb, "nb.csv", output_format="csv", pages='all')

# Prince Edward Island

html = requests.get(pe).text
pe_soup = BeautifulSoup(html, 'html.parser')
pe_data = pe_soup.findAll("div", class_="even")[0].findAll('li')


In [6]:
# Read the Nova Scotia CSV file

with open("ns.csv", newline="") as file:
    ns_reader = csv.reader(file)
    # Data format: [[],[], ... , []]
    ns_data = list(ns_reader)

In [7]:
# Read the New Brunswick CSV file

with open("nb.csv", newline="") as file:
    nb_reader = csv.reader(file)
    # Data format: [[],[], ... , []]
    nb_data = list(nb_reader)

In [8]:
# Nova Scotia

# List where the name will be localy stored
ns_list = []


index = 0
while(index != len(ns_data)):
    
    # Remove the string from its array
    string = ns_data[index][0]
    
    # Example of name: 
    #
    # useless o/a Name - Address
    ns_first_split = string.split('o/a ')
    
    # If it has 'o/a' in the text
    if(len(ns_first_split) == 1):
        name_address = ns_first_split[0]
    elif(len(ns_first_split) == 2):
        name_address =  ns_first_split[1]
        
    # Name - Address
    ns_second_split = name_address.split(' - ')
    if(len(ns_second_split) == 2):
        name = ns_second_split[0]
        address = ns_second_split[1]
        item = {"name": name,"address":address}
        #ns_list[index] = item
        ns_list.append(item)
    else:
        name = ns_second_split[0]
        item = {"name": name}
        #ns_list['list'][index] = item
        ns_list.append(item)
        
    index = index+1

In [9]:
# New Brunswick

# List where the name will be localy stored
nb_list = []


index = 0
while(index != len(nb_data)):
    
    # Remove the string from its array
    string = nb_data[index][0]
    
    # Example of name: 
    #
    # Name (Address)
    nb_split = string.split(' (')
    
    if(len(nb_split) == 2):
        name = nb_split[0]
        address = nb_split[1]
        item = {"name": name,"address":address[:-1]}
        nb_list.append(item)
    else:
        name = nb_split[0]
        item = {"name": name}
        nb_list.append(item)
        
    index = index+1

In [10]:
# Prince Edward Island
pe_list = []

for li in pe_data:
    # Remove the string from its array
    string = li.text
    #print(string)
    
    # Example of name: 
    #
    # 000000 Name
    if(string[0].isdigit()):
        pe_split = string.split(' ')
        if(len(pe_split)>1):
            index = 2
            name = pe_split[1] + ' '
            while(index != len(pe_split)):
                if(index !=len(pe_split)):
                    name = name + pe_split[index]+' '
                else:
                    name = name + pe_split[index]
                index = index+1
            item = {"name": name}
            pe_list.append(item)
    else:
        item = {"name": string}
        pe_list.append(item)

In [11]:
# Saving JSON

#---------------------- NEED TO SORT THE JSON ----------------------#

with open('designated-employers.json', 'w') as file:
    json.dump({"results": {"nb": nb_list, "ns": ns_list, "pe":pe_list, "nl": nl_list}}, file)
    
    